# BayesianInference.jl Usage Example

This notebook demonstrates the basic usage of the `BayesianInference.jl` package.

## Setup

First, activate the package environment and load the necessary packages.

In [ ]:
using Pkg
# Activate the package environment located in the parent directory
Pkg.activate(joinpath(@__DIR__, ".."))
# Install dependencies if needed
Pkg.instantiate()

using BayesianInference
using PythonCall

## Initialize BayesInference

Create an instance of the BI object with your desired configuration.

In [4]:
# Initialize the BI object
m = importBI(platform="cpu")

Python: <BI.Main.main.bi object at 0x7761d4b4e990>

## Generate Synthetic Data

Create some sample data for demonstration.

In [10]:
# True parameters
alpha = 0.2
beta = 0.6
sigma = 1.2

# Generate data
x = m.dist.normal(0, 1, shape=(100,), sample=true)
y = m.dist.normal(alpha + beta * x, sigma, sample=true)

m.data_on_model =  pydict(Dict("x" => x, "y" => y))

Python:
{'x': Array([-0.23604248, -0.606091  , -1.17281575,  0.32558339,  0.10901318,
        0.44182044,  1.761894  ,  1.46962104,  1.38529584, -0.20473494,
        0.82020252,  0.33599178, -1.4625772 , -0.47964699, -0.73672936,
        1.5071632 ,  0.46673293,  1.91547267,  0.80970302,  0.68639909,
       -0.98568394, -0.17355008, -1.52959954,  0.76896304,  1.24612033,
       -0.72760383, -0.51133283, -0.10486702,  1.60429085, -0.84496755,
        1.04883198,  1.04204983, -0.42929814,  0.90504335, -0.23383239,
       -0.20443847,  0.44981699, -1.0682377 , -0.04225268, -2.67462318,
        0.1856763 ,  0.44611415,  0.9677847 , -0.831991  ,  0.20944074,
       -0.23985909, -1.67258007,  0.68974566, -1.83442427, -0.8461506 ,
                              ... 25 more lines ...
        1.34365413e+00, -2.21436959e-01, -2.60318871e-01,  1.53260077e+00,
       -1.35635400e+00, -1.02882105e+00, -2.21764431e+00, -6.05891078e-01,
       -6.34651630e-02, -1.54056713e+00, -1.05677933e+00, -2.811

## Define the Bayesian Model

Create a function that specifies the probabilistic model.

In [4]:
# Define the model
function model(; x, y)
    # Priors
    alpha = m.dist.normal(loc=0, scale=1, name="alpha")
    beta  = m.dist.normal(loc=0, scale=1, name="beta")
    sigma = m.dist.exponential(1, name="sigma")

    # Likelihood
    mu = alpha + beta * x
    m.dist.normal(mu, sigma, obs=y)
end

model (generic function with 1 method)

## Fit the Model

Run MCMC sampling to fit the model to the data.

In [11]:
# Fit the model
m.fit(model, num_warmup=1000, num_samples=1000, num_chains=1)

jax.local_device_count 32


sample: 100%|##########| 2000/2000 [00:00<00:00, 2578.78it/s, 7 steps of size 7.43e-01. acc. prob=0.92]


Python: None

## View Results

Display the posterior summary statistics.

In [12]:
# Display summary
m.summary()

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.18,0.13,-0.05,0.36,0.0,0.00,858.27,757.00,NaN
beta,0.69,0.13,0.50,0.89,0.0,0.01,910.18,573.14,NaN
sigma,1.28,0.09,1.11,1.39,0.0,0.00,839.83,676.94,NaN
